In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [ ]:
train_path = r'../input/dog-breed-identification/train/'
valid_path = r'../input/dog-breed-identification/test/'

In [ ]:
resnet = ResNet50(
    include_top=False,
    weights="imagenet")

In [ ]:
resnet.summary()

In [ ]:
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
#Flatten last but one layer
x = GlobalAveragePooling2D()(resnet.output)

In [ ]:
#Connect last but one layer to 120 prediction nodes
prediction = Dense(120, activation='softmax')(x)

In [ ]:
# create a model object
custom_resnet = Model(inputs=resnet.input, outputs=prediction)

In [ ]:
#Complie the model
custom_resnet.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
custom_resnet.summary()

In [ ]:
#Now that the bare model is ready, do data preparation
#https://vijayabhaskar96.medium.com/tutorial-on-keras-flow-from-dataframe-1fd4493d237c

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

def append_ext(fn):
    return fn+".jpg"

traindf=pd.read_csv(r"/kaggle/input/dog-breed-identification/labels.csv",dtype=str)
testdf=pd.read_csv(r"/kaggle/input/dog-breed-identification/sample_submission.csv",dtype=str)
traindf["id"]=traindf["id"].apply(append_ext)
testdf["id"]=testdf["id"].apply(append_ext)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.25)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_dataframe(dataframe=traindf,
                                                 directory=train_path,
                                                 x_col="id",
                                                 y_col="breed",
                                                 shuffle=True,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')



In [ ]:
valid_set=train_datagen.flow_from_dataframe(dataframe=traindf,
                                                 directory=train_path,
                                                 x_col="id",
                                                 y_col="breed",
                                                 shuffle=True,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

In [ ]:
#Fit the custom resnet model with above dataset
r = custom_resnet.fit(
  training_set,
  validation_data=valid_set,
  epochs=100,
  steps_per_epoch=len(training_set),
  validation_steps=len(valid_set)
)

In [ ]:
import matplotlib.pyplot as plt

# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
custom_resnet.save('dog_classification.h5')